In [ ]:
seed = 42
%run -i /home/jupyter/MathVGerrmandering_CMAT_2021/common_v4.py
states = get_states()
for abbr in ['RI', 'TX']:
    for level in ['tract', 'bg', 'tabblock']:
        overwrite = []
        gerry = Gerry(abbr, level=level, census_yr=2010)
        gerry.get_data(overwrite)
# overwrite = [
#     'crosswalks',
#     'assignments',
# #     'elections',
    
# #     'shapes_tract',
# #     'shapes_bg',
# #     'shapes_tabblock',
# #     'edges_tract',
# #     'edges_bg',
# #     'edges_tabblock',
#     'census',
#     ]

# gerry.make_graph()

In [247]:
seed = 42
%run -i /home/jupyter/MathVGerrmandering_CMAT_2021/common_v6.py
states = get_states()
overwrite = [
#     'crosswalks',
#     'assignments',
#     'elections',
#     'census',
#     'shapes',
#     'nodes',
#     'edges',
#     'graph',
    ]

g = Gerry('RI', level='bg', district='cd')
g.get_data(overwrite=overwrite)



Get Rhode Island crosswalks tabblock 2020   ... BigQuery table exists ... success

Get Rhode Island assignments tabblock 2020  ... zip already exists ... processing ... success

Get Rhode Island elections cntyvtd 2020     ... elections only implemented for TX ... success

Get Rhode Island census tabblock 2010       ... BigQuery table exists ... success

Get Rhode Island shapes bg 2020             ... BigQuery table exists ... success

Get Rhode Island edges bg 2020              ... BigQuery table exists ... success

Get Rhode Island nodes bg 2020              ... BigQuery table exists ... success

Get Rhode Island graph bg 2020 cd           ... pickle file already exists ... success


In [163]:
N = g.nodes
N['geoid_2020'].value_counts().max()
N[N.duplicated(subset='geoid_2020', keep=False)]

,geoid_2020,district,pop_total,area,perim,S


In [141]:
N = g.nodes

i = N.columns.drop(['party', 'votes', 'candidate', 'office', 'yr'], errors='ignore').to_list()

N = N.pivot_table(index=i, columns='party', values='votes', fill_value=0).reset_index()
N.columns.name = None
N

,geoid_2020,cntyvtd_2020,cd,sldu,sldl,pop_total,area,perim,D,L,R
0,48001950100,001000016,5,3,8,4685.000000,483306613,111477.444690,278.272002,11.837349,1554.808049
1,48001950401,001000010,5,3,8,5422.000000,16509268,24137.613946,19.738449,0.759171,175.368524
2,48001950402,001000010,5,3,8,7343.741841,71134275,61904.266063,118.215553,1.597507,395.383099
3,48001950500,001000007,5,3,8,4377.000000,23132052,26815.530240,412.703505,9.055825,633.097003
4,48001950600,001000007,5,3,8,6405.000000,20653882,29500.160687,552.014312,12.157886,1111.557131
...,...,...,...,...,...,...,...,...,...,...,...
7392,48507950200,507000001,23,19,80,1880.000000,590775035,150370.786498,362.504923,0.000000,189.120926
7393,48507950200,507000004,23,19,80,1880.000000,590775035,150370.786498,0.000000,3.945061,0.000000
7394,48507950301,507000001,23,19,80,2254.000000,1635811105,231490.066644,443.001836,0.000000,103.388387
7395,48507950301,507000002,23,19,80,2254.000000,1635811105,231490.066644,0.000000,0.891968,0.000000


In [ ]:

        print('making graph', end=concat_str)
        g = f'geoid_{yr}'
        self.edges = read_table(self.table_id('edges', self.level, yr))

        district_types = {'cd', 'sldu', 'sldl'}
        district_types.remove(self.district)
        self.nodes = (read_table(self.table_id('nodes', self.level, yr))
                      .drop(columns=district_types)
                      .rename(columns={self.district:'district'})
                     )
        districts = np.unique(self.nodes['district'])
        self.graph = self.edges_to_graph(self.edges)
        nx.set_node_attributes(self.graph, self.nodes.set_index(g).to_dict('index'))

        print('connecting districts', end=concat_str)
        shapes_tbl = self.table_id('shapes', self.level, yr)
        for dist, nodes in self.nodes.groupby('district')[g]:
            while True:
                H = self.graph.subgraph(nodes)
                components = sorted([list(c) for c in nx.connected_components(H)], key=lambda x:len(x), reverse=True)
                if len(components) == 1:
                    break
                print(f'\nDistrict {str(dist).rjust(3, " ")} has {str(len(components)).rjust(3, " ")} connected components with {[len(c) for c in components]} nodes ... adding edges to connect', end=concat_str)
                c = ["', '".join(components[i]) for i in range(2)]
                query = f"""

In [101]:
(g.nodes['geoid_2020'].value_counts() == 4)

g.nodes.query("geoid_2020 == '48439123200'")
# mask = g.nodes['geoid_2020'].value_counts()>1
# g.nodes[g.nodes.duplicated(subset='geoid_2020', keep=False)]

party,geoid_2020,cntyvtd_2020,cd,sldu,sldl,pop_total,area,perim,D,L,R
7484,48439123200,439004573,12,10,90,0.0,7259043,12935.619924,0.000000,0.000000,0.000000
7483,48439123200,439001001,12,10,90,1896.0,7259043,12935.619924,0.000000,0.000000,250.021794
7481,48439123200,439001001,12,10,90,1042.0,7259043,12935.619924,0.000000,13.006997,0.000000
7482,48439123200,439001001,12,10,90,1677.0,7259043,12935.619924,540.340799,0.000000,0.000000


In [71]:
g.nodes

party,geoid_2020,cntyvtd_2020,cd,sldu,sldl,pop_total,area,perim,D,L,R
0,480019501001000,001000018,5,3,8,52.000000,4899115,12387.183187,3.184376,0.137703,15.835816
1,480019501001001,001000018,5,3,8,124.916983,1738542,8383.995339,7.649666,0.330796,38.041584
2,480019501001002,001000018,5,3,8,3.000000,65303,1616.835917,0.183714,0.007944,0.913605
3,480019501001003,001000018,5,3,8,0.000000,654,241.487188,0.000000,0.000000,0.000000
4,480019501001004,001000018,5,3,8,3.000000,22188,597.258571,0.183714,0.007944,0.913605
...,...,...,...,...,...,...,...,...,...,...,...
668751,485079503025020,507000002,23,19,80,54.000000,7280,391.645072,11.217717,0.067374,2.206488
668752,485079503025021,507000002,23,19,80,111.000000,33161,789.735853,23.058640,0.138490,4.535558
668753,485079503025022,507000002,23,19,80,12.000000,6361,315.023016,2.492826,0.014972,0.490331
668754,485079503025023,507000002,23,19,80,19.000000,8334,371.136952,3.946974,0.023706,0.776357


In [45]:
N = g.N.copy().reset_index(drop=True)
# N['perim'] = N['perim'].round(2)
i = N.columns.drop(['party', 'votes', 'candidate', 'office', 'yr', 'area', 'perim'], errors='ignore').to_list()
# N = N.set_index(i)
#                     .pivot_table(columns='party', values='votes')
# N
# print(i)
# # if abbr == 'TX':
N = N.pivot(index=i, columns='party', values='votes').fillna(0).reset_index()
# N.groupby(i).sum()
# N = N.pivot_table(index=i, columns='party', values='votes', fill_value=0).reset_index()
# N = # i.to_list()
# N.unstack('party')
N

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
select
    substring(A.{geoid}, 1, {g}) as {geoid},
    A.{geoid} as {geoid}_tabblock,
    B.{cntyvtd},
    A.cntyvtd_pop_prop,
    cast(B.cd_{self.shapes_yr} as int) as cd,
    cast(B.sldu_{self.shapes_yr} as int) as sldu,
    cast(B.sldl_{self.shapes_yr} as int) as sldl,
    A.total as pop_total
from
    {self.table_id('census', 'tabblock', self.census_yr)} as A
inner join
    {self.table_id('assignments', 'tabblock', self.shapes_yr)} as B
on
    A.{geoid} = B.{geoid}

In [22]:
N.isnull().any(axis=1).sum()

0

In [23]:
N

party,geoid_2020,cntyvtd_2020,cntyvtd_pop_prop,cd,sldu,sldl,pop_total,area,perim,D,L,R
0,480019501001000,001000018,0.017213,5.0,3.0,8.0,52.000000,4899115,12387.183187,185,8,920
1,480019501001001,001000018,0.041350,5.0,3.0,8.0,124.916983,1738542,8383.995339,185,8,920
2,480019501001002,001000018,0.000993,5.0,3.0,8.0,3.000000,65303,1616.835917,185,8,920
3,480019501001003,001000018,0.000000,5.0,3.0,8.0,0.000000,654,241.487188,185,8,920
4,480019501001004,001000018,0.000993,5.0,3.0,8.0,3.000000,22188,597.258571,185,8,920
...,...,...,...,...,...,...,...,...,...,...,...,...
668751,485079503025020,507000002,0.016843,23.0,19.0,80.0,54.000000,7280,391.645072,666,4,131
668752,485079503025021,507000002,0.034623,23.0,19.0,80.0,111.000000,33161,789.735853,666,4,131
668753,485079503025022,507000002,0.003743,23.0,19.0,80.0,12.000000,6361,315.023016,666,4,131
668754,485079503025023,507000002,0.005926,23.0,19.0,80.0,19.000000,8334,371.136952,666,4,131


In [2]:
N = g.N.copy()
# N['party'].fillna('L', inplace=True)
# N['votes'].fillna(0, inplace=True)
# mask = N.isnull().any(axis=1)
N[mask]

NameError: name 'g' is not defined

In [1]:
1+1

2

In [54]:
N = g.N.reset_index()
# print(N.shape)
# # g.N['votes'].unique()
# # g.N
# i = N.columns.drop(['party', 'votes', 'candidate', 'office', 'yr'], errors='ignore')
# if g.abbr == 'TX':
#     N = N.pivot(index=i, columns='party', values='votes').fillna(0)#.reset_index()


In [35]:
N

,,,,,,,,,party,NaN,D,L,R
index,geoid_2020,cntyvtd_2020,cntyvtd_pop_prop,cd,sldu,sldl,pop_total,area,perim,,,,
0,480410020192008,041000010,0.002946,17.0,5.0,14.0,15.505127,22314,692.996598,0.0,863.0,0.0,0.0
1,480410020192008,041000010,0.002946,17.0,5.0,14.0,15.505127,22314,692.996598,0.0,0.0,30.0,0.0
2,480410020192008,041000010,0.002946,17.0,5.0,14.0,15.505127,22314,692.996598,0.0,0.0,0.0,1233.0
3,480410020141006,041000029,0.000000,17.0,5.0,14.0,0.000000,1497,166.697544,0.0,191.0,0.0,0.0
4,480410020141006,041000029,0.000000,17.0,5.0,14.0,0.000000,1497,166.697544,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1967730,480291618021039,029001094,0.002544,20.0,19.0,117.0,5.826845,15645,613.027259,0.0,0.0,5.0,0.0
1967731,480291618021039,029001094,0.002544,20.0,19.0,117.0,5.826845,15645,613.027259,0.0,0.0,0.0,213.0
1967732,480291620011029,029001146,0.002919,23.0,19.0,117.0,1.092675,27795,805.182161,0.0,36.0,0.0,0.0


In [26]:
g.N[g.N.isnull().any(axis=1)]
g.N

,geoid_2020,cntyvtd_2020,cntyvtd_pop_prop,cd,sldu,sldl,pop_total,party,votes,area,perim
0,480410020192008,041000010,0.002946,17.0,5.0,14.0,15.505127,D,863.0,22314,692.996598
1,480410020192008,041000010,0.002946,17.0,5.0,14.0,15.505127,L,30.0,22314,692.996598
2,480410020192008,041000010,0.002946,17.0,5.0,14.0,15.505127,R,1233.0,22314,692.996598
3,480410020141006,041000029,0.000000,17.0,5.0,14.0,0.000000,D,191.0,1497,166.697544
4,480410020141006,041000029,0.000000,17.0,5.0,14.0,0.000000,L,5.0,1497,166.697544
...,...,...,...,...,...,...,...,...,...,...,...
1967730,480291618021039,029001094,0.002544,20.0,19.0,117.0,5.826845,L,5.0,15645,613.027259
1967731,480291618021039,029001094,0.002544,20.0,19.0,117.0,5.826845,R,213.0,15645,613.027259
1967732,480291620011029,029001146,0.002919,23.0,19.0,117.0,1.092675,D,36.0,27795,805.182161
1967733,480291620011029,029001146,0.002919,23.0,19.0,117.0,1.092675,L,1.0,27795,805.182161


In [5]:
select
    C.*,
    D.party,
    D.votes * C.cntyvtd_pop_prop as votes
    --D.votes * C.cntyvtd_pop_prop as votes
from (
    {query}
    ) as C
left join (
    select
        *
    from
        {self.table_id('elections', 'cntyvtd', self.shapes_yr)}
    where
        race = "{self.race}" and
        office = "{self.race_office}" and
        yr = {self.race_yr}
    ) as D
on
    C.cntyvtd_{self.shapes_yr} = D.cntyvtd_{self.shapes_yr}
    """


271298

In [42]:
c = f'cntyvtd_2020'
d = g.elect[~g.elect[c].isin(g.assign.values)]
d.sort_values('votes', ascending=False)
d['orig'].value_counts()

32500001d    221
32500003d    221
32500002a    221
32500004c    221
32500002c    221
            ... 
31100004a    131
03300003b    127
31100002b    126
03300002b    121
03700005b     13
Name: orig, Length: 163, dtype: int64

In [41]:
s ='2770000'
mask = g.assign.str[:len(s)] == s
g.assign[mask]

5548      27700001B
5552      27700001C
5561      27700001F
5562      27700001H
5587      27700002D
5718      27700002E
5771      27700002F
5829      27700002G
5908      27700003E
5909      27700003F
5941      27700003G
474265    27700001D
474280    27700001E
474286    27700003D
474425    27700003I
474482    27700004B
474549    27700004C
474570    27700004D
474581    27700004E
474598    27700004F
474640    27700004G
521019    27700001G
521089    27700001I
Name: cntyvtd_2020, dtype: object

In [ ]:
mask = g.N.isnull().any(axis=1)
g.N[mask]

In [ ]:
g.N.isnull().sum(axis=1).value_counts()

In [ ]:
m = g.N.isnull().any(axis=1)
g.N[m]

In [ ]:
df=pd.read_csv('/home/jupyter/redistricting_data/assignments/tabblock/2020/TX/BlockAssign_ST48_TX_VTD.txt', sep='|')

In [ ]:
df

In [ ]:
cv = df['DISTRICT']
# cv.str.len().unique()
cv.str[2].value_counts()
# .value_counts()
# cv[cv<10].shape

In [ ]:
e = read_table(g.table_id('elections', 'cntyvtd', 2020))

In [8]:
e = read_table('cmat-315920.redistricting_data.elections_cntyvtd_2020_TX')
e['party'].unique()

array(['D', 'G', 'L', 'R'], dtype=object)

In [9]:
e[e['party'].isnull()]

,index,county,fips,vtd,cntyvtd,office,name,party,incumbent,votes,yr,race,cntyvtd_2020


In [7]:
e = pd.read_csv('/home/jupyter/redistricting_data/elections/cntyvtd/2020/TX/2020_General_Election_Returns.csv')
a = pd.read_csv('/home/jupyter/redistricting_data/assignments/tabblock/2020/TX/BlockAssign_ST48_TX_VTD.txt', sep='|')

In [ ]:
a

In [ ]:
e['Par']

In [ ]:
df = pd.DataFrame()
df['b'] = b['d'].str[-2].value_counts()
df['f'] = f['d'].str[-2].value_counts()
df

In [ ]:
b = b.apply(lambda x:x.astype(str).str.upper())
b

In [ ]:
elect

In [ ]:
def f(df):
    cols = [x.lower() for x in df.columns]
    cols[1] = 'cnty'
    cols[2] = 'vtd'
    df.columns = cols
    df['cnty'] = df['cnty'].astype(str).str.lower()
    df['vtd']  = df['vtd'] .astype(str).str.lower()
    df['cntyvtd']  = df['cnty'].str.rjust(3, '0') + df['vtd'].str.rjust(6, '0')
    df['cntyvtd2'] = df['cnty'].str.rjust(3, '0') + df['vtd'].str[:-1].str.rjust(6, '0')
    return df

elect  = f(pd.read_csv('/home/jupyter/redistricting_data/elections/cntyvtd/2020/TX/2020_General_Election_Returns.csv')).query('votes > 0')
assign = f(pd.read_csv('/home/jupyter/redistricting_data/assignments/tabblock/2020/TX/BlockAssign_ST48_TX_VTD.txt', sep='|'))



In [ ]:
# pd.read_csv('/home/jupyter/redistricting_data/assignments/tabblock/2020/TX/BlockAssign_ST48_TX_VTD.txt', sep='|')
pd.read_csv('/home/jupyter/redistricting_data/elections/cntyvtd/2020/TX/2020_General_Election_Returns.csv').dtypes

In [ ]:
a = assign['cntyvtd'].drop_duplicates()
unmatched = ~elect['cntyvtd'].isin(a)
print(unmatched.sum())
elect.loc[unmatched, 'cntyvtd'] = elect.loc[unmatched, 'cntyvtd2']
unmatched = ~elect['cntyvtd'].isin(a)
print(unmatched.sum())
elect.drop(columns='cntyvtd2', inplace=True)
elect


In [ ]:
np.unique(assign.query("cnty=='113'")['vtd'])

In [ ]:

elect  = pd.read_csv('/home/jupyter/redistricting_data/elections/cntyvtd/2020/TX/2020_General_Election_Returns.csv')
elect.columns = [x.lower for x in elect]
assign = pd.read_csv('/home/jupyter/redistricting_data/assignments/tabblock/2020/TX/BlockAssign_ST48_TX_VTD.txt', sep='|').apply(w, axis=0)

def f(df):
    w = lambda x:x.astype(str).str.lower()
    return df.apply(w, axis=0).rename(columns={x:x.lower for x in df.columns})

def f(df):
    
    df = df.apply(w, axis=0).drop_duplicates()
    df.columns = ['cty', 'vtd']
    df['geoid'] = df['cty'].str.rjust(3, '0') + df['vtd'].str.rjust(6, '0')
    return df

a = f(assign[['COUNTYFP', 'DISTRICT']])
e = f(elect[['FIPS', 'VTD']])

# elect = elect[['COUNTYFP', 'DISTRICT']].apply(w, axis=0).drop_duplicates()
# elect.columns = ['cty', 'vtd']
# elect['geoid'] = elect['cty'].str.rjust(3, '0') + elect['vtd'].str.rjust(6, '0')

# assign = assign[['FIPS', 'VTD']].apply(w, axis=0).drop_duplicates()
# assign.columns = ['cty', 'vtd']
# assign['geoid'] = elect['cty'].str.rjust(3, '0') + elect['vtd'].str.rjust(6, '0')
X = e.merge(a[['geoid', 'cty']], on='geoid', how='left', suffixes=('','_y'))
mask = X.isnull().any(axis=1)
Y = X[mask].copy()
Y['new_geoid'] = Y['cty'].str.rjust(3, '0') + Y['vtd'].str[:-1].str.rjust(6, '0')
set(Y['new_geoid']).difference(a['geoid'])
# display(X[mask]['vtd_x'].str[-1].value_counts())
# display(X[~mask]['vtd_x'].str[-1].value_counts())
# Y
# f['e'] = (f['c'].astype(str).str.rjust(3, '0') + f['d'].astype(str).str.rjust(6, '0')).str.upper()
# mask = f['d'].str.len() > 4
# f[mask]
# f
# D = set(b['e']).symmetric_difference(f['e'])
# np.unique([d[-2] for d in D])



In [ ]:
# elect  = pd.read_csv('/home/jupyter/redistricting_data/elections/cntyvtd/2020/TX/2020_General_Election_Returns.csv')
# assign = pd.read_csv('/home/jupyter/redistricting_data/assignments/tabblock/2020/TX/BlockAssign_ST48_TX_VTD.txt', sep='|')

def f(df):
    w = lambda x:x.astype(str).str.upper()
    df = df.apply(w, axis=0).drop_duplicates()
    df.columns = ['cty', 'vtd']
    df['geoid'] = df['cty'].str.rjust(3, '0') + df['vtd'].str.rjust(6, '0')
    return df

a = f(assign[['COUNTYFP', 'DISTRICT']])
e = f(elect[['FIPS', 'VTD']])

# elect = elect[['COUNTYFP', 'DISTRICT']].apply(w, axis=0).drop_duplicates()
# elect.columns = ['cty', 'vtd']
# elect['geoid'] = elect['cty'].str.rjust(3, '0') + elect['vtd'].str.rjust(6, '0')

# assign = assign[['FIPS', 'VTD']].apply(w, axis=0).drop_duplicates()
# assign.columns = ['cty', 'vtd']
# assign['geoid'] = elect['cty'].str.rjust(3, '0') + elect['vtd'].str.rjust(6, '0')
X = e.merge(a[['geoid', 'cty']], on='geoid', how='left', suffixes=('','_y'))
mask = X.isnull().any(axis=1)
Y = X[mask].copy()
Y['new_geoid'] = Y['cty'].str.rjust(3, '0') + Y['vtd'].str[:-1].str.rjust(6, '0')
set(Y['new_geoid']).difference(a['geoid'])
# display(X[mask]['vtd_x'].str[-1].value_counts())
# display(X[~mask]['vtd_x'].str[-1].value_counts())
# Y
# f['e'] = (f['c'].astype(str).str.rjust(3, '0') + f['d'].astype(str).str.rjust(6, '0')).str.upper()
# mask = f['d'].str.len() > 4
# f[mask]
# f
# D = set(b['e']).symmetric_difference(f['e'])
# np.unique([d[-2] for d in D])



In [ ]:
Y[Y['new_geoid'] == "113000465"]

In [ ]:
mask = b['e'].str[-3:] == '4/6'
b[mask]

In [ ]:
mask = ~f['e'].str[-2].str.isnumeric()
f[mask]

In [ ]:
1+1

In [ ]:
a['DISTRICT'].str[-2].value_counts()
mask = a['DISTRICT'].str[-2] == '-'
a[mask]

In [ ]:
# a
mask = (a['COUNTYFP']==339)
pd.set_option("display.max_rows", 200)

a[mask]['DISTRICT'].value_counts()

In [ ]:
# mask = ge['cntyvtd'] == '3390038A'
mask = (ge['Name'] == 'Young') & (ge['Votes'] == 3744)
ge[mask]

In [ ]:
a = read_table(g.table_id('assignments', 'tabblock', 2020))
e = read_table('cmat-315920.redistricting_data.elections_cntyvtd_2020_TX')

In [ ]:
col='cntyvtd_2020'
# display(a[col].str[-1].value_counts())
display(e[col].str[-2].value_counts())
# mask = a[col].str == '3390038'
# display(a[mask])
# mask = e[col].str.upper() == '3390038A'
# e[mask]

# e[col].str[0].value_counts()

# mask = ~a[col].str[-1].str.isnumeric()
# # mask = a['cntyvtd_2020'] == '3970004'
# a[mask].sort_values(col)

In [ ]:
a[~mask]

In [ ]:
mask = e.isnull().any(axis=1)
e[mask]

In [ ]:
mask.mean()
e['party'].value_counts()

In [ ]:
f = e.query("party in ['R', 'D', 'L', 'G']")
mask = f.isnull().any(axis=1)
f[mask]

In [ ]:
g.make_graph()
g.graph.nodes(data=True)

In [ ]:
g.graph.nodes(data=True)

In [ ]:
gerry.get_nodes()
N = gerry.nodes.sort_values(['geoid_2020'])


In [ ]:
        url = f"https://www2.census.gov/geo/docs/maps-data/data/baf2020/BlockAssign_ST{self.fips}_{self.abbr}.zip"
        L = []
        zipfile = fetch_zip(url, self.file_id(variable))
        for fn in zipfile.namelist():
            col = fn.lower().split('_')[-1][:-4]
            if fn[-3:] == 'txt' and col != 'aiannh':
                df = extract_file(zipfile, fn, sep='|')
                if col == 'vtd':
                    df['countyfp'] = df['countyfp'] + df['district'].str[-4:]
                    col = 'cntyvtd'
                df = df.iloc[:,:2]
                df.columns = [f'geoid_2020', f'{col}_2020']
                L.append(df.set_index(f'geoid_2020'))
        df = pd.concat(L, axis=1).reset_index()
        load_table(tbl, df=df)

In [ ]:
gerry.nodes

In [ ]:
import google, time, datetime, dataclasses, typing, os, pathlib, shutil, urllib, zipfile as zf, numpy as np, pandas as pd, geopandas as gpd, networkx as nx
import matplotlib.pyplot as plt, plotly.express as px
from shapely.ops import orient
from google.cloud import aiplatform, bigquery
try:
    from google.cloud.bigquery_storage import BigQueryReadClient
except:
    os.system('pip install --upgrade google-cloud-bigquery-storage')
    from google.cloud.bigquery_storage import BigQueryReadClient

import warnings
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
warnings.filterwarnings('ignore', message='.*Pyarrow could not determine the type of columns*')
pd.set_option('display.max_columns', None)

proj_id = 'cmat-315920'
cred, proj = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient   = bigquery.Client(credentials=cred, project=proj)
root_path  = '/home/jupyter'
root_path  = pathlib.Path(root_path)
data_path  = root_path / 'redistricting_data'
bq_dataset = proj_id   +'.redistricting_data'
rng = np.random.default_rng(seed)

def lower_cols(df):
    df.rename(columns = {x:str(x).lower() for x in df.columns}, inplace=True)
    return df

def extract_file(zipfile, fn, **kwargs):
    file = zipfile.extract(fn)
    return lower_cols(pd.read_csv(file, dtype=str, **kwargs))

def read_table(tbl, rows=99999999999, start=0, cols='*'):
    query = f'select {", ".join(cols)} from {tbl} limit {rows}'
    if start is not None:
        query += f' offset {start}'
    return bqclient.query(query).result().to_dataframe()

def head(tbl, rows=10):
    return read_table(tbl, rows)

def load_table(tbl, df=None, file=None, query=None, overwrite=True, preview_rows=0):
    if overwrite:
        bqclient.delete_table(tbl, not_found_ok=True)
    if df is not None:
        job = bqclient.load_table_from_dataframe(df, tbl).result()
    elif file is not None:
        with open(file, mode="rb") as f:
            job = bqclient.load_table_from_file(f, tbl, job_config=bigquery.LoadJobConfig(autodetect=True)).result()
    elif query is not None:
        job = bqclient.query(query, job_config=bigquery.QueryJobConfig(destination=tbl)).result()
    else:
        raise Exception('at least one of df, file, or query must be specified')
    if preview_rows > 0:
        display(head(tbl, preview_rows))
    return tbl

def fetch_zip(url, file):
    path = file.parent
    path.mkdir(parents=True, exist_ok=True)
    os.chdir(path)
    try:
        zipfile = zf.ZipFile(file)
        print(f'zip already exists', end='')
    except:
        try:
            print(f'fetching zip from {url} ', end = '')
            zipfile = zf.ZipFile(urllib.request.urlretrieve(url, file)[0])
            print(f'done', end = '')
        except urllib.error.HTTPError:
            print('\n\nFAILED - BAD URL\n\n')
            zipfile = None
    return zipfile

def get_states():
    query = f"""
    select
        state_fips_code as fips
        , state_postal_abbreviation as abbr
        , state_name as name
    from
        bigquery-public-data.census_utility.fips_codes_states
    where
        state_fips_code <= '56'
    """
    return lower_cols(bqclient.query(query).result().to_dataframe())

def yr_to_congress(yr):
    return min(116, int(yr-1786)/2)

@dataclasses.dataclass
class Gerry:
    # These are default values that can be overridden when you create the object
    abbr              : str
    level             : str = 'tract'
    census_yr         : int = 2010
    shapes_yr         : int = 2020
    district          : str = 'cd'
    race              : str = 'General_Election'
    race_office       : str = 'U.S. Sen'
    race_yr           : int = 2018
    chunk_size        : int = 50000
    min_graph_degree  : int = 1
    pop_err_max_pct   : float = 2.0
    clr_seq           : typing.Any = tuple(px.colors.qualitative.Antique)
    
    def __getitem__(self, key):
        return self.__dict__[key]

    def __setitem__(self, key, val):
        self.__dict__[key] = val
        
    def __post_init__(self):
        levels = ['tract', 'bg', 'tabblock']
        assert self.level in levels, f"level must be one of {levels}, got {self.level}"
        district_types = ['cd', 'sldu', 'sldl']
        assert self.district in district_types, f"district must be one of {district_types}, got {self.district}"
        census_yrs = list(range(2010, 2021))
        assert self.census_yr in census_yrs, f"census_yr must be one of {census_yrs}, got {self.census_yr}"
        shapes_yrs = [2010, 2010]
        assert self.shapes_yr in shapes_yrs, f"census_yr must be one of {shapes_yrs}, got {self.shapes_yr}"
        
        self.__dict__.update(states[states['abbr']==self.abbr].iloc[0])
        def rgb_to_hex(c):
            if c[0] == '#':
                return c
            else:
                return '#%02x%02x%02x' % tuple(int(rgb) for rgb in c[4:-1].split(', '))
        self.clr_seq = [rgb_to_hex(c) for c in self.clr_seq]

    def table_id(self, variable, level, yr):
        tbl = f"{bq_dataset}.{variable}_{level}_{yr}_{self.abbr}"
        return tbl
    
    def file_id(self, variable=None, level=None, yr=None, tbl=None, suffix='zip'):
        if tbl is None:
            tbl = self.table_id(variable, level, yr)
        f = tbl.split('.')[-1]
        return data_path / f"{f.replace('_','/')}/{f}.{suffix}"

    def get_assignments(self, tbl):
        variable, level, yr, abbr = tbl.split('.')[-1].split('_')
        url = f"https://www2.census.gov/geo/docs/maps-data/data/baf{self.shapes_yr}/BlockAssign_ST{self.fips}_{self.abbr}.zip"
        L = []
        zipfile = fetch_zip(url, self.file_id(tbl=tbl))
        for fn in zipfile.namelist():
            col = fn.lower().split('_')[-1][:-4]
            if fn[-3:] == 'txt' and col != 'aiannh':
                df = extract_file(zipfile, fn, sep='|')
                if col == 'vtd':
                    df['countyfp'] = df['countyfp'] + df['district'].str[-4:]
                    col = 'cntyvtd'
                df = df.iloc[:,:2]
                df.columns = [f'geoid_{self.shapes_yr}', f'{col}_{self.shapes_yr}']
                L.append(df.set_index(f'geoid_2020'))
        df = pd.concat(L, axis=1).reset_index()
        load_table(tbl, df=df)


    def get_crosswalks(self, tbl):
        variable, level, yr, abbr = tbl.split('.')[-1].split('_')
        url = f"https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST{self.fips}.zip"
        zipfile = fetch_zip(url, self.file_id(tbl=tbl))
        for fn in zipfile.namelist():
            df = extract_file(zipfile, fn, sep='|')
            for y in [2010, 2020]:
                df[f'geoid_{y}'] = df[f'state_{y}'].str.rjust(2,'0') + df[f'county_{y}'].str.rjust(3,'0') + df[f'tract_{y}'].str.rjust(6,'0') + df[f'blk_{y}'].str.rjust(4,'0')
        load_table(tbl, df=df)                


    def get_elections(self, tbl):
        variable, level, yr, abbr = tbl.split('.')[-1].split('_')
        if abbr != 'TX':
            print(f'elections only implemented for TX', end='')
            return
        url = f'https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/{self.shapes_yr}-general-vtd-election-data.zip'
        L = []
        zipfile = fetch_zip(url, self.file_id(tbl=tbl))
        for fn in zipfile.namelist():
            w = fn.split('_')
            if w.pop(-1) == 'Returns.csv':
                df = extract_file(zipfile, fn, sep=',')
                df[f'cntyvtd_{yr}'] = df['cntyvtd'].str.rjust(7, '0')
                df.drop(columns=['cntyvtd'], inplace=True)
                df['yr'] = w.pop(0)
                df['race'] = '_'.join(w)
                L.append(df)
        df = pd.concat(L, axis=0, ignore_index=True).astype({'votes':int, 'yr':int})
        load_table(tbl, df=df)


    def get_census(self, tbl):
        variable, level, yr, abbr = tbl.split('.')[-1].split('_')
        url = f"https://www2.census.gov/programs-surveys/decennial/{self.shapes_yr}/data/01-Redistricting_File--PL_94-171/{self.name.replace(' ', '_')}/{self.abbr.lower()}{yr}.pl.zip"
        zipfile = fetch_zip(url, self.file_id(tbl=tbl))
        for fn in zipfile.namelist():
            if fn[-3:] == '.pl':
                print(f'{concat_str}{fn}', end='')
                file = zipfile.extract(fn)
                if fn[2:5] == 'geo':
                    geo_tbl  = tbl + 'geo'
                    temp_tbl = tbl + 'temp'
                    load_table(temp_tbl, file=file)
                    sel = [f'trim(substring(string_field_0, {s}, {w})) as {n}' for s, w, n in zip(census_columns['starts'], census_columns['widths'], census_columns['geo'])]
                    query = 'select\n\t' + ',\n\t'.join(sel) + '\nfrom\n\t' + temp_tbl
                    load_table(geo_tbl, query=query)
                    bqclient.delete_table(temp_tbl)
                else:
                    i = fn[6]
                    if i in ['1', '2']:
                        cmd = 'sed -i "1s/^/' + ','.join(census_columns['joins'] + census_columns[i]) + '\\n/" ' + file
                        os.system(cmd)
                        load_table(tbl+i, file=file)

        print(f"{concat_str}joining", end='' )
        t = ',\n    '
        query = f"""
select
    concat(right(concat("00",C.state), 2), right(concat("000",C.county), 3), right(concat("000000",C.tract), 6), right(concat("0000",C.block), 4)) as geoid_{self.shapes_yr}
    , C.*
    , A.{f'{t}A.'.join(census_columns['1'])}
    , B.{f'{t}B.'.join(census_columns['2'])}
from
    {tbl}1 as A
inner join
    {tbl}2 as B
on
    A.fileid = B.fileid
    and A.stusab = B.stusab
    and A.logrecno = B.logrecno
inner join
    {geo_tbl} as C
on
    A.fileid = trim(C.fileid)
    and A.stusab = trim(C.stusab)
    and A.logrecno = cast(C.logrecno as int)
where
    C.block != ""
    """

        if self.census_yr == 2010:
            query = f"""
select
    E.geoid_{self.shapes_yr}
    , E.area_prop
    , D.*
from (
    {query}
    ) as D
inner join (
    select
        case when area_{self.census_yr} > 0.1 then area_int / area_{self.census_yr} else 0 end as area_prop
        , *
    from (
        select
            geoid_{self.census_yr}
            , geoid_{self.shapes_yr}
            , cast(arealand_int as int) as area_int
            , sum(cast(arealand_int as int)) over (partition by geoid_2010) as area_{self.census_yr}
        from
            {self.table_id('crosswalks', 'tabblock', self.census_yr)}
        )
    ) as E
on
    D.geoid_2010 = E.geoid_2010
    """

            query = f"""
select
    geoid_{self.shapes_yr}
    , sum(area_prop) as area_prop
    , {t.join([f'max({c}) as {c}'             for c in census_columns['geo']])}
    , {t.join([f'sum(area_prop * {c}) as {c}' for c in census_columns['1'] + census_columns['2']])}
from (
    {query}
    )
group by
    1
    """

        query = f"""
select
    case when cntyvtd_pop > 0 then total / cntyvtd_pop else 1 / cntyvtd_blocks end as cntyvtd_pop_prop
    , *
from (
    select
        sum(total) over (partition by cntyvtd_2020) as cntyvtd_pop
        , count(*) over (partition by cntyvtd_2020) as cntyvtd_blocks
        , cntyvtd_{self.shapes_yr}
        , F.*
    from (
        {query}
        ) as F
    inner join
        {self.table_id('assignments', 'tabblock', self.shapes_yr)} as G
    on
        F.geoid_2020 = G.geoid_2020
    )
    """
        load_table(tbl, query=query)
        bqclient.delete_table(geo_tbl)
        bqclient.delete_table(tbl+'1')
        bqclient.delete_table(tbl+'2')


    def get_shapes(self, tbl):
        variable, level, yr, abbr = tbl.split('.')[-1].split('_')
        temp_tbl = tbl + '_temp'
        url = f"https://www2.census.gov/geo/tiger/TIGER{self.shapes_yr}/{self.level.upper()}"
        if self.shapes_yr == 2010:
            url += '/2010'
        elif self.shapes_yr == 2020 and self.level == 'tabblock':
            url += '20'
        url += f"/tl_{self.shapes_yr}_{self.fips}_{self.level}{str(self.shapes_yr)[-2:]}"
        if self.shapes_yr == 2020 and self.level in ['tract', 'bg']:
            url = url[:-2]
        url += '.zip'

        file = self.file_id(tbl=tbl)
        path = file.parent
        zipfile = fetch_zip(url, file)
        zipfile.extractall(path)

        a = 0
        while True:
            print(f'Starting row {a}')
            df = lower_cols(gpd.read_file(path, rows=slice(a, a+self.chunk_size)))
            df.columns = [x[:-2] if x[-2:].isnumeric() else x for x in df.columns]
            df = df[['geoid', 'aland', 'awater', 'intptlon', 'intptlat', 'geometry']].rename(columns={'geoid':f'geoid_{self.shapes_yr}'})
            df['geometry'] = df['geometry'].apply(lambda p: orient(p, -1))
            load_table(temp_tbl, df=df.to_wkb(), overwrite=a==0)
            if df.shape[0] < self.chunk_size:
                break
            else:
                a += self.chunk_size

        query = f"""
select
    geoid_{self.shapes_yr}
    , aland
    , awater
    , perim
    , case when perim > 0 then 4 * acos(-1) * aland / (perim * perim) else 0 end as polsby_popper
    , point
    , geography
from (
    select
        *
        , st_perimeter(geography) as perim
    from (
        select
            *
            , st_geogpoint(cast(intptlon as float64), cast(intptlat as float64)) as point
            , st_geogfrom(geometry) as geography
        from
            {temp_tbl}
        )
    )
order by
    geoid_self.shapes_yr
    """
        load_table(tbl, query=query)
        bqclient.delete_table(temp_tbl)


    def get_edges(self, tbl):
        variable, level, yr, abbr = tbl.split('.')[-1].split('_')
        shapes_tbl = self.table_id('shapes', level, yr)
        t = ',\n        '
        query = f"""
select
    *
from (
    select
        x.geoid_{self.shapes_yr} as geoid_{self.shapes_yr}_x,
        y.geoid_{self.shapes_yr} as geoid_{self.shapes_yr}_y,
        st_distance(x.point, y.point) as distance,
        st_length(st_intersection(x.geography, y.geography)) as shared_perim
    from
        {shapes_tbl} as x,
        {shapes_tbl} as y
    where
        x.geoid_{self.shapes_yr} < y.geoid_{self.shapes_yr} and st_intersects(x.geography, y.geography)
    )
where shared_perim > 0.1
    """
        load_table(tbl, query=query)
    self.edges = read_table(tbl)
    print('success', end='')


    def get_nodes(self, tbl):
        variable, level, yr, abbr = tbl.split('.')[-1].split('_')
        if abbr != 'TX':
            print(f'elections only implemented for TX', end='')
            return
        
        query = f"""
select
    A.geoid_{self.shapes_yr}
    , B.party
    , B.votes * A.cntyvtd_pop_prop as votes
    , A.total as pop_total
from
    {self.table_id('census', self.level, self.census_yr)} as A
inner join (
    select
        *
    from
        {self.table_id('elections', self.level, self.shapes_yr)}
    where
        race = "{self.race}"
        and office = "{self.race_office}"
        and yr = {self.race_yr}
    ) as B
on
    A.cntyvtd_{self.shapes_yr} = B.cntyvtd_{self.shapes_yr}
    """

        if self.level != 'tabblock':
            g = 12 if self.level == 'bg' else 11
            query = f"""
select
    substring(geoid_{self.shapes_yr}, 1, {g}) as geoid_{self.shapes_yr}
    , party
    , sum(votes) as votes
    , sum(pop_total) as pop_total
from (
    {query}
    )
group by
    1, 2
   """
        query = f"""
select
    A.*
    , B.aland as area
    , B.perim
from (
    {query}
    ) as A
inner join
    {self.table_id('shapes', self.level, self.shapes_yr)} as B
on
    A.geoid_{self.shapes_yr} = B.geoid_{self.shapes_yr}
    """
        N = bqclient.query(query).result().to_dataframe()
        i = N.columns.drop(['party', 'votes', 'candidate', 'office', 'yr'], errors='ignore')
        N = N.pivot(index=i, columns='party', values='votes').reset_index()
        load_table(tbl, df=N)
    self.nodes = read_table(tbl)
    print('success', end='')


    def make_graph(self):
        self.graph = nx.from_pandas_edgelist(self.edges, source='geoid_2020_x', target='geoid_2020_y', edge_attr=['distance', 'shared_perim'])
        nx.set_node_attributes(self.graph, self.nodes.set_index('geoid_2020').to_dict('index'))


    def get_data(self, overwrite=list()):
        for variable, level, yr in [
            ('crosswalks', 'tabblock', self.shapes_yr),
            ('assignments', 'tabblock', self.shapes_yr),
            ('elections', 'cntyvtd', self.shapes_yr),
            ('census', 'tabblock', self.census_yr),
            ('shapes', self.level, self.shapes_yr),
            ('edges', self.level, self.shapes_yr),
            ('nodes', self.level, self.shapes_yr),
        ]:
            print(f"\nGet {self.name} {variable} {level} {yr}".ljust(20, ' '), end=concat_str)
            tbl = self.table_id(variable, level, yr)
            try:
                assert variable not in overwrite
                bqclient.get_table(tbl)
                print('BigQuery table exists', end='')
            except:
                getattr(self, f'get_{variable}')(tbl)
            

#         yr = 2020
#         variable = 'assignments'
#         print(f"\nGet {self.name} {variable.ljust(15, ' ')} {yr}", end=concat_str)
#         self.get_assignments(variable in overwrite)
        
#         yr = 2020
#         variable = 'elections'
#         print(f"\nGet {self.name} {variable.ljust(15, ' ')} {yr}", end=concat_str)
#         self.get_elections(variable in overwrite)

#         yr = self.census_yr
#         variable = 'census'
#         print(f"\nGet {self.name} {variable.ljust(15, ' ')} {yr}", end=concat_str)
#         self.get_census(variable in overwrite)

#         yr = 2020
#         variable = f'shapes_{self.level}'
#         print(f"\nGet {self.name} {variable.ljust(15, ' ')} {yr}", end=concat_str)
#         self.get_shapes(variable in overwrite)

#         yr = 2020
#         variable = f"edges_{self.level}"
#         print(f"\nGet {self.name} {variable.ljust(15, ' ')} {yr}", end=concat_str)
#         self.get_shapes(self.level, variable in overwrite)

      

        
        
        
        
#         if L == 'tabblock':
#             g = 15
#         elif L == 'bg':
#             g = 12
#         elif L == 'tract':
#             g = 11
        
# #         nodes = read_table(self.table_id(f"shapes_{self.level}"))
#         query = f"""
# select
#     *
# from (
#     select
#         substring(A.geoid_2020, 1, {g}) as geoid_2020
#         , sum(total)
#     from
#         {self.table_id('census')}
#     group by
#         1
#     ) as A
# inner join
#     {self.table_id(f"shape_{L}")}
# """
#         node_attrs = ['geoid_2020', 'cntyvtd_pop_prop', 'total']
#         nodes = read_table(self.table_id('census'), cols=node_attrs)
#         display(nodes.head(3))
        
#         nx.set_node_attributes(self['graph'], nodes.set_index('geoid_2020').to_dict('index'))
#         print(self['graph'].nodes(data=True))

            

#         return 

#     def read_shape_pairs(self, level='tabblock', yr=2020, rows=None, start=None, cols='*'):
#         tbl = self.table_id(f'shapes_{level}', yr)
#         df = read_table(tbl, rows, start, cols)
#         for c in ['centroid', 'geometry', 'geometry_simple']:
#             if c in df.columns:
#                 df[c] = gpd.GeoSeries.from_wkb(df[c])
#         return gpd.GeoDataFrame(df, geometry='geometry', crs=crs_map)

                
                
            
concat_str = ' ... '
census_columns = {
    'joins':  ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno'],

    'widths': [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18],

    'geo': ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved'],
                  
    '1': ['total', 'population_of_one_race', 'white_alone', 'black_or_african_american_alone', 'american_indian_and_alaska_native_alone', 'asian_alone', 'native_hawaiian_and_other_pacific_islander_alone', 'some_other_race_alone', 'population_of_two_or_more_races', 'population_of_two_races', 'white_black_or_african_american', 'white_american_indian_and_alaska_native', 'white_asian', 'white_native_hawaiian_and_other_pacific_islander', 'white_some_other_race', 'black_or_african_american_american_indian_and_alaska_native', 'black_or_african_american_asian', 'black_or_african_american_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_some_other_race', 'american_indian_and_alaska_native_asian', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander', 'asian_some_other_race', 'native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_three_races', 'white_black_or_african_american_american_indian_and_alaska_native', 'white_black_or_african_american_asian', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_some_other_race', 'white_american_indian_and_alaska_native_asian', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander', 'white_asian_some_other_race', 'white_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_asian_some_other_race', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_asian_some_other_race', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_four_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_asian_some_other_race', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_asian_some_other_race', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_five_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_six_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_hl', 'hispanic_or_latino_hl', 'not_hispanic_or_latino_hl', 'population_of_one_race_hl', 'white_alone_hl', 'black_or_african_american_alone_hl', 'american_indian_and_alaska_native_alone_hl', 'asian_alone_hl', 'native_hawaiian_and_other_pacific_islander_alone_hl', 'some_other_race_alone_hl', 'population_of_two_or_more_races_hl', 'population_of_two_races_hl', 'white_black_or_african_american_hl', 'white_american_indian_and_alaska_native_hl', 'white_asian_hl', 'white_native_hawaiian_and_other_pacific_islander_hl', 'white_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_hl', 'black_or_african_american_asian_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_some_other_race_hl', 'american_indian_and_alaska_native_asian_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_hl', 'asian_some_other_race_hl', 'native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_three_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_hl', 'white_black_or_african_american_asian_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_asian_some_other_race_hl', 'white_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_asian_some_other_race_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_asian_some_other_race_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_four_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_asian_some_other_race_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_five_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_six_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl'],

    '2': ['total_18', 'population_of_one_race_18', 'white_alone_18', 'black_or_african_american_alone_18', 'american_indian_and_alaska_native_alone_18', 'asian_alone_18', 'native_hawaiian_and_other_pacific_islander_alone_18', 'some_other_race_alone_18', 'population_of_two_or_more_races_18', 'population_of_two_races_18', 'white__black_or_african_american_18', 'white__american_indian_and_alaska_native_18', 'white__asian_18', 'white__native_hawaiian_and_other_pacific_islander_18', 'white__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native_18', 'black_or_african_american__asian_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__some_other_race_18', 'american_indian_and_alaska_native__asian_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander_18', 'asian__some_other_race_18', 'native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_three_races_18', 'white__black_or_african_american__american_indian_and_alaska_native_18', 'white__black_or_african_american__asian_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__some_other_race_18', 'white__american_indian_and_alaska_native__asian_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander_18', 'white__asian__some_other_race_18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__asian__some_other_race_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__asian__some_other_race_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_four_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__asian__some_other_race_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__asian__some_other_race_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_five_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_six_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'total_hl18', 'hispanic_or_latino_hl18', 'not_hispanic_or_latino_hl18', 'population_of_one_race_hl18', 'white_alone_hl18', 'black_or_african_american_alone_hl18', 'american_indian_and_alaska_native_alone_hl18', 'asian_alone_hl18', 'native_hawaiian_and_other_pacific_islander_alone_hl18', 'some_other_race_alone_hl18', 'population_of_two_or_more_races_hl18', 'population_of_two_races_hl18', 'white__black_or_african_american_hl18', 'white__american_indian_and_alaska_native_hl18', 'white__asian_hl18', 'white__native_hawaiian_and_other_pacific_islander_hl18', 'white__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native_hl18', 'black_or_african_american__asian_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__some_other_race_hl18', 'american_indian_and_alaska_native__asian_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander_hl18', 'asian__some_other_race_hl18', 'native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_three_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native_hl18', 'white__black_or_african_american__asian_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__asian__some_other_race_hl18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__asian__some_other_race_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__asian__some_other_race_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_four_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__asian__some_other_race_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_five_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_six_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'housing_total', 'housing_occupied', 'housing_vacant'],
}

census_columns['starts'] = 1 + np.insert(np.cumsum(census_columns['widths'])[:-1], 0, 0)

In [ ]:
tbl = g.table_id('assignments2')
bqclient.get_table(tbl)

In [ ]:
d = N.columns.drop(['party', 'votes', 'candidate', ''], errors='ignore')
P = N.pivot(index=d,columns='party', values='votes').reset_index()
P.head(5)

In [ ]:
P.reset_index()

In [ ]:
P['sum'] = P['D'] + P['R'] + P['L']


In [ ]:
# tbl = gerry.table_id('census', 2010)
# df = read_table(tbl)
N.groupby('cntyvtd_2020')['cntyvtd_pop_prop'].sum().round(4)#.value_counts()

In [ ]:
d = N.columns.drop(['party', 'votes', 'votes_orig'])
P = N.pivot(index=d,columns='party', values='votes')
P.head(5)

In [ ]:
P['D'] + P['R'] + P['L']

In [ ]:
N.sort_values('geoid_2020')

In [ ]:
N = gerry['nodes']
# N['candidate'].unique()

In [ ]:
# tbl = gerry.table_id('census', 2010)
# df=read_table(tbl)
N.groupby('cntyvtd_2020')['cntyvtd_pop_prop'].sum().sort_values()

In [ ]:
M = N.groupby(['cntyvtd_2020','party'])['cntyvtd_pop_prop'].sum().round(4)#.sort_values('geoid_2020')#
M.min(), M.max()

In [ ]:
select
    A.geoid_2020
    , A.cntyvtd_2020
    , A.total as pop_total
    --, B.office
    --, B.yr
    , B.party
    --, A.cntyvtd_pop_prop
    --, C.name as candidate
    , B.votes * A.cntyvtd_pop_prop as votes
    --, B.votes as votes_orig
from
    {self.table_id('census', self['census_yr'])} as A
inner join (
    select
        *
    from
        {self.table_id('elections')}
    where
        race = "{self['race']}"
        and office = "{self['race_office']}"
        and yr = {self['race_yr']}
    ) as B
on
    A.cntyvtd_2020 = B.cntyvtd_2020
)

In [ ]:
tbl = gerry.table_id('census', 2010)
tbl
c = read_table(tbl)

In [ ]:
c['cntyvtd_block'] = c.groupby('cntyvtd_2020').transform('count')

In [ ]:
c

In [ ]:
def f(x):
    return x['votes'].sum() - x['votes_orig'].max()
# N.groupby('cntyvtd_2020')['cntyvtd_pop_prop'].sum().sort_values()
# N.groupby(['cntyvtd_2020','party'])['votes'].sum()#.sort_values('geoid_2020')#.apply(f)
M = N.groupby(['cntyvtd_2020','party']).apply(f)#.sort_values('geoid_2020')#
M.min(), M.max()

In [ ]:
W = M.round()
W.value_counts()

In [ ]:
gerry['nodes'].sort_values('geoid_2020')

In [ ]:
pd.pivot?

In [ ]:
select
    B.geoid_2020
    , A.cntyvtd_2020
    , A.yr as election_yr
    , A.race as contest
    , A.office as office
    , A.name as candidate
    , A.party as party
    , A.votes * B.cntyvtd_pop_prop as votes
    , A.votes as votes_cntyvtd
from
    {self.table_id('elections')} as A
inner join
    {self.table_id('census')} as B
on
    A.cntyvtd_2020 = B.cntyvtd_2020

In [ ]:
select
    *
    , C.votes * B.cntyvtd_pop_prop as votes
from
    {self.table_id(f"shapes_tabblock")} as A
left join
    {self.table_id('census')} as B
on
    A.geoid_2020 = B.geoid_2020
left join (
    select
        *
    from
        {self.table_id('elections')}
    where
        race = {self['race']}
        and office = {self['race_office']}
        and yr = {self['race_yr']}
    ) as C

In [ ]:
gerry

In [ ]:
gerry.get_elections()

In [ ]:
self = gerry
read_table(self.table_id('pairs'))

In [ ]:
G = gerry.make_graph()

In [ ]:
# def read_tbl(tbl, rows=1e15, start=0, cols='*'):
#     query = f'select {cols} from {tbl} limit {rows}'
#     if start is not None:
#         query += f' offset {start}'
#     try:
#         return bqclient.query(query).result().to_dataframe()
#     except:
#         return None
    
# tbl = gerry.table_id('census')
A = gerry.table_id('census')
B = gerry.table_id(f"shapes_tabblock", 2020)

query = f"""
select
    A.geoid_2020,
    B.geoid_2020,
from
    {A} as A
full outer join
    {B} as B
on
    A.geoid_2020 = B.geoid_2020
where
    A.geoid_2020 is Null or B.geoid_2020 is Null
"""

df = bqclient.query(query).result().to_dataframe()
df
# print(A,B)

In [ ]:
pairs = read_table(gerry.table_id('pairs_bg', 2020), 10)

In [ ]:
pairs

In [ ]:
df.isnull().any()

In [ ]:
dfA = read_table(B, rows=5)
dfA

In [ ]:
dfA

In [ ]:
#             query = f"""
# select
#     *,
#     --shared_perim > 0.1 as touches
# from (
#     select
#         x.geoid_{yr} as geoid_{yr}_x,
#         y.geoid_{yr} as geoid_{yr}_y,
#         st_intersects(x.geography, y.geography)
#         --x.geography as geography_x,
#         --st_distance(x.point, y.point) as distance,
#         --st_length(st_intersection(x.geography, y.geography)) as shared_perim
#     from
#         {shapes_tbl} as x,
#         {shapes_tbl} as y
#     where
#         x.geoid_{yr} < y.geoid_{yr}
#     )
# order by
#     1, 2
#     """

In [ ]:
    select
        *
    from
        {self.table_id('elections')}
    where
        race = "{self['race']}"
        and office = "{self['race_office']}"
        and yr = {self['race_yr']}
    ) as C


In [ ]:
x=815
# x=900000
int(x*(x-1)/2)

In [ ]:
331705

In [ ]:
w = '\n\t, '.join([f'{T}.{col} as {col}_{T}' for col in ['geoid_2020', 'point', 'geography'] for T in ['A', 'B']])
print(w)

In [ ]:
import pygeos

In [ ]:
! pip install pygeos

In [ ]:
import pygeos

In [ ]:
def fetch_zip(self, url, variable, yr=2020):
    z = self.file_id(variable, yr, 'zip')
    path = z.parent
    path.mkdir(parents=True, exist_ok=True)
    os.chdir(path)
    try:
        zipfile = zf.ZipFile(z)
        print(f'zip already exists', end='')
    except:
        try:
            print(f'fetching zip from {url}', end = '')
            zipfile = zf.ZipFile(urllib.request.urlretrieve(url, z)[0])
        except urllib.error.HTTPError:
            print('\n\nFAILED - BAD URL\n\n')
            zipfile = None
    return zipfile


In [ ]:
import google, time, datetime, dataclasses, typing, os, pathlib, shutil, urllib, zipfile as zf, numpy as np, pandas as pd, geopandas as gpd, networkx as nx
import matplotlib.pyplot as plt, plotly.express as px
from shapely.ops import orient
from google.cloud import aiplatform, bigquery
try:
    from google.cloud.bigquery_storage import BigQueryReadClient
except:
    os.system('pip install --upgrade google-cloud-bigquery-storage')
    from google.cloud.bigquery_storage import BigQueryReadClient

import warnings
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
warnings.filterwarnings('ignore', message='.*Pyarrow could not determine the type of columns*')
pd.set_option('display.max_columns', None)

proj_id = 'cmat-315920'
cred, proj = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient   = bigquery.Client(credentials=cred, project=proj)

tbl = f'{proj_id}.redistrcting_data.bg_2020_Ri



In [ ]:
tbl = gerry.table_id('tract', 2010)
tbl
df = head(tbl, 10)
# df = gpd.GeoDataFrame.from_wkb(head(tbl, 10))

# dftry'] = gpd.GeoSeries.from_wkb(df['geometry'])
for col in ['centroid', 'geometry', 'geometry_simple']:
    df[col] = gpd.GeoSeries.from_wkb(df[col])
# df['geometry'] = gpd.GeoSeries.from_wkb(df['geometry'])
df = gpd.GeoDataFrame(df, geometry='geometry', crs=crs_map)
df.sort_values('aland')
# df.plot()
df

In [ ]:
gerry.tract.to_wkt?

In [ ]:
z = gerry.tract
z.write()

In [ ]:
        tbl = self.table_id(variable, yr)
        if not overwrite and head(tbl) is not None:
            print(f'{concat_str}{yr} BigQuery table exists', end='')
        else:
            url = f"https://www2.census.gov/geo/docs/maps-data/data/baf"
            if yr == 2020:
                url += '2020'
            url += f"/BlockAssign_ST{self['fips']}_{self['abbr']}.zip"
            L = []
            zipfile = self.fetch_zip(url, variable, yr)
            for fn in zipfile.namelist():
                col = fn.lower().split('_')[-1][:-4]
                if fn[-3:] == 'txt' and col != 'aiannh':
                    if yr == 2020:
                        sep = '|'
                    else:
                        sep = ','
                    df = extract_file(zipfile, fn, sep=sep)
                    if col == 'vtd':
                        df['countyfp'] = df['countyfp'] + df['district'].str[-4:]
                        col = 'cntyvtd'
                    df = df.iloc[:,:2]
                    df.columns = [f'geoid_{yr}', f'{col}_{yr}']
                    L.append(df.set_index(f'geoid_{yr}'))
            df = pd.concat(L, axis=1).reset_index()
            self.load_table(variable, yr, df=df, overwrite=overwrite)